# MathpixConverter with LlamaIndex RAG 📚🦙

This notebook demonstrates a simple workflow for converting papers from PDF to plain text, and then setting up a query engine on top of converted documents.

Since PDF conversion involves replacing tables and diagrams with text, we're going to need to set up an LLM and a VLM to do that. We're going to be using OpenAI's GPT-4 and GPT-4V.

## 1. Environment

Start by setting necessary environment variables in the `.env` file.

- `MATHPIX_APP_ID`
- `MATHPIX_APP_KEY`
- `OPENAI_API_KEY`

In [ ]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv(".env"))  # read local .env file

In [ ]:
from pathlib import Path

## 2. Converter 

`MathpixPdfConverter` uses LangChain abstractions to interact with LLMs. It takes a text and a vision model in its constructor.
Note that the prompts were tuned to work with GPT-4.

In [ ]:
from llama_index.llms import OpenAI
from llama_index.multi_modal_llms import OpenAIMultiModal
from pdf_processor.core import MathpixPdfConverter

In [ ]:
text_model = OpenAI()
vision_model = OpenAIMultiModal(max_new_tokens=4096)

In [ ]:
converter = MathpixPdfConverter(text_model=text_model, vision_model=vision_model)

We're going to set up an `inbox` directory to store our unprocessed PDFs, as well as `fulltext` directory to record converter's outputs.
Then, we're going to convert PDFs one by one.

The converter outputs a `PdfResult` object that contains the final text in the `.content` field, as well as intermediate result of the conversion.

In [ ]:
inbox_path = Path("./inbox")
fulltext_path = Path("./fulltext")
fulltext_path.mkdir(exist_ok=True, parents=True)

In [ ]:
for pdf_path in inbox_path.glob("*.pdf"):
    pdf_result = converter.convert(pdf_path)
    with fulltext_path.joinpath(f"{pdf_path.stem}.txt").open("w") as f:
        f.write(pdf_result.content)

## 3. Query Engine

Now let's set up a basic LlamaIndex Query Engine to be able to query our converted documents. Inside, the `rag_factory` follows this [Starter Tutorial](https://docs.llamaindex.ai/en/stable/getting_started/starter_example.html).

In [ ]:
from rag_factory import build_query_engine

In [ ]:
query_engine = build_query_engine(fulltext_path.as_posix())

In [ ]:
response = query_engine.query("How do action unit activations correspond to stress?")
print(response)

## 4. Evaluation

We're going to calculate the RAG triad of metrics using TruLens integration with LlamaIndex.

In [ ]:
from trulens_eval import Tru, Feedback, TruLlama
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider import OpenAI as TruLensOpenAI

import numpy as np

In [ ]:
tru = Tru()
tru.reset_database()

In [ ]:
provider = TruLensOpenAI(model_engine="gpt-4")
context = TruLlama.select_source_nodes().node.text  # select context to provide it to the feedback function

We're going to define a feedback function for each of the metrics.

In [ ]:
# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(provider.relevance_with_cot_reasons).on_input_output()

In [ ]:
grounded = Groundedness(groundedness_provider=provider)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect())  # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

In [ ]:
f_context_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

Come up with queries to calculate the scores on, and run evaluation.

In [ ]:
tru_recorder = TruLlama(
    query_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_context_relevance,
        f_groundedness
    ]
)

In [ ]:
eval_questions = [
    "How do action unit activations correspond to stress?",
]

In [ ]:
for question in eval_questions:
    with tru_recorder as recording:
        query_engine.query(question)

Display the results.

In [ ]:
tru.get_leaderboard(app_ids=[])

In [ ]:
tru.run_dashboard()